In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import grad

from Heston_NN import Net,weights_init

In [2]:
save_net = True
#save_net = False

In [3]:
table = pd.read_csv('Heston_data_input')
X = table.drop(['C_price','delta'], axis=1)
y = table[['C_price']]
#X = df[['k','T','C_price']]
#y = df[['v0','rho','kappa','theta','sigma']]
T_loc = list(X.columns).index('T')
k_loc = list(X.columns).index('k') # log strike 

In [4]:
table['C_price'].describe()

count    63.000000
mean      0.285844
std       0.134396
min       0.005952
25%       0.251236
50%       0.289855
75%       0.381712
max       0.452502
Name: C_price, dtype: float64

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.2, random_state=42)
input_scaler = preprocessing.MinMaxScaler()
output_scaler = preprocessing.MinMaxScaler()
X_train_scaled = input_scaler.fit_transform(X_train)
#y_train_scaled = output_scaler.fit_transform(y_train)
#y_train_scaled = y_train # no scaling
#X_train_scaled = X_train_scaled.reshape(X_train_scaled.shape[0],X_train_scaled.shape[1],1) # for lstm
X_test_scaled = input_scaler.transform(X_test)
#y_test_scaled = y_test # no scaling

#X_train_scaled = torch.FloatTensor(X_train_scaled,requires_grad=True)
#X_test_scaled = torch.FloatTensor(X_test_scaled,requires_grad=True)
X_train_scaled = torch.FloatTensor(X_train_scaled)
X_train_scaled.requires_grad = True
X_test_scaled = torch.FloatTensor(X_test_scaled)
X_test_scaled.requires_grad = True
y_train = torch.FloatTensor(y_train.values)
y_test = torch.FloatTensor(y_test.values)

In [9]:
model = Net()
model.apply(weights_init)
        
print(model)

Net(
  (fc1): Linear(in_features=7, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=128, bias=True)
  (fc7): Linear(in_features=128, out_features=1, bias=True)
)


# Training

In [ ]:
arbitrage_weight = [0.1,0.1]; l2_weight = 1e-5
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-3)
epochs = 10001
#epochs = 1001
loss_arr = []

for i in range(epochs):
    if i % 500 == 0:
        print('Epoch: ',i,' ')
        show_log = True
    else:
        show_log = False
 
    loss = model.get_loss(X_train_scaled, y_train, T_loc, k_loc, criterion, arbitrage_weight, l2_weight, show_log=show_log)
    loss_arr.append(loss)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    


Epoch:  0  
calendar  34   tensor(3.6248, grad_fn=<AddBackward0>)
butterfly  1   tensor(0.1021, grad_fn=<AddBackward0>)
Loss: 3.7599334716796875
Epoch:  500  
Loss: 0.0015573782147839665
Epoch:  1000  
Loss: 0.0013559574726969004
Epoch:  1500  
Loss: 0.001200366416014731
Epoch:  2000  
Loss: 0.001066751778125763
Epoch:  2500  
Loss: 0.0009480195585638285
Epoch:  3000  
Loss: 0.0008380989311262965
Epoch:  3500  
Loss: 0.0007372391410171986
Epoch:  4000  
Loss: 0.0006443352904170752
Epoch:  4500  
Loss: 0.0005568615160882473
Epoch:  5000  
Loss: 0.00047937192721292377
Epoch:  5500  
Loss: 0.00041167830931954086
Epoch:  6000  
Loss: 0.00035545791615732014
Epoch:  6500  
Loss: 0.0003088088415097445
Epoch:  7000  
Loss: 0.00027191900881007314
Epoch:  7500  
Loss: 0.00024471725919283926
Epoch:  8000  
Loss: 0.00022658854140900075
Epoch:  8500  
Loss: 0.0002158898423658684


In [ ]:
if save_net: torch.save(model.state_dict(), "heston_NN") # save the model
plt.plot(loss_arr)
plt.yscale('log')

In [ ]:
m2 = Net()
m2.load_state_dict(torch.load('heston_NN'))
m2.eval() # evaluation mode

# Test case

In [ ]:
loss = model.get_loss(X_test_scaled, y_test, T_loc, k_loc, criterion, arbitrage_weight, l2_weight, show_log=True)

# autograd

In [ ]:
x = X_train_scaled[0].clone().detach()
x.requires_grad = True
y= model.forward(x)
dydx = grad(y,x, create_graph = True)[0]

In [ ]:
if dydx[T_loc] < 0.0:
    print('nono ',torch.exp(dydx[T_loc] ))

In [ ]:
d2ydx2 = grad(dydx[T_loc],x, create_graph = True)[0]

In [ ]:
d2ydx2

In [ ]:
x = X_train_scaled.clone().detach()
x.requires_grad = True
y= model.forward(x)

In [ ]:
calendar_loss = torch.tensor(0.)
butterfly_loss = torch.tensor(0.)
for elem in X_train_scaled:
    y= model.forward(elem)
    dydx = grad(y, elem, create_graph = True)[0]
    dydT = dydx[T_loc]
    if dydT < 0.0:
        calendar_arbi_count += 1
        calendar_loss += torch.exp(-dydT[T_loc]) * 1e-1
    dydk = dydx[k_loc] # dCdk w.r.t. log-strike 
    d2ydk2 = grad(dydx[k_loc],elem, create_graph = True)[0][k_loc] # d2Cdk2 w.r.t. log-strike
    # dCdK = 1/K*dcdk
    # d2CdK2 = 1/e^2k * (d2Cdk2 - dCdk)
    # butterfly arbitrage: d2CdK2 = 1/K^2(d2Cdk2 - dCdk) > 0, d2Cdk2 > dCdk
    if d2ydk2 - dydk < 0.0: # violation of butterfly arbitrage
        butterfly_loss += torch.exp(-(d2ydk2 - dydk)) * 1e-1

In [ ]:
"""
class Net(nn.Module):

    def __init__(self,num_input=7, num_neurons=128):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(num_input, num_neurons) 
        self.fc2 = nn.Linear(num_neurons, num_neurons)
        self.fc3 = nn.Linear(num_neurons, num_neurons)
#        self.fc4 = nn.Linear(num_neurons, num_neurons)
#        self.fc5 = nn.Linear(num_neurons, num_neurons)
#        self.fc6 = nn.Linear(num_neurons, num_neurons)
        self.fc7 = nn.Linear(num_neurons, 1)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.relu(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = F.tanh(self.fc3(x))
#        x = F.relu(self.fc3(x))
#        x = F.relu(self.fc4(x))
#        x = F.relu(self.fc5(x))
#        x = F.relu(self.fc6(x))
        x = F.relu(self.fc7(x))
        return x
    
    def get_loss(self, x, y_train, T_loc, k_loc, criterion, arbitrage_weight, l2_weight, show_log=False):
        y_hat = self.forward(x)
        loss = criterion(y_hat, y_train)
        l2_reg = torch.tensor(0.)

        # Penalization (loop over each data)
        ## calendar arbitrage
        calendar_arbi_count = 0
        calendar_loss = torch.tensor(0.)
        butterfly_loss = torch.tensor(0.)
        for elem in x:
            y= self.forward(elem)
            dydx = grad(y, elem, create_graph = True)[0]
            dydT = dydx[T_loc]
            if dydT < 0.0:
                calendar_arbi_count += 1
                calendar_loss += torch.exp(-dydT) * arbitrage_weight[0]
        loss += calendar_loss
        ## butterfly arbitrage
        butterfly_count = 0
        dydk = dydx[k_loc] # dCdk w.r.t. log-strike 
        d2ydk2 = grad(dydx[k_loc],elem, create_graph = True)[0][k_loc] # d2Cdk2 w.r.t. log-strike
        # butterfly arbitrage: d2CdK2 = 1/K^2(d2Cdk2 - dCdk) > 0, d2Cdk2 > dCdk
        # dCdK = 1/K*dcdk
        # d2CdK2 = 1/e^2k * (d2Cdk2 - dCdk)
        butter_ineq = (d2ydk2 - dydk)/torch.exp(2.0*elem[k_loc])
        if butter_ineq < 0.0: # violation of butterfly arbitrage
            butterfly_count += 1
            butterfly_loss += torch.exp(-butter_ineq) * arbitrage_weight[1]
        loss += butterfly_loss

        # regularizations
        for param in self.parameters():
            l2_reg += torch.norm(param)
            loss += l2_lambda * l2_reg
            
        if show_log:
            if calendar_arbi_count > 0: print('calendar ',i, ' ',calendar_arbi_count,' ',calendar_loss)
            if butterfly_count > 0: print('butterfly ',i, ' ',butterfly_count,' ',butterfly_loss)
            print(f'Loss: {loss}')
            
        return loss
        

def weights_init(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
#        torch.nn.init.normal_(m.weight)
#        xavier(m.weight.data)
#        xavier(m.bias.data)

"""

In [ ]:
"""
epochs = 30001
#epochs = 101
loss_arr = []
optimizer = optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-3)
l2_lambda = 1e-5
criterion = nn.MSELoss()


for i in range(epochs):
    y_hat = model.forward(X_train_scaled)
    loss = criterion(y_hat, y_train)
    l2_reg = torch.tensor(0.)
    
    # Penalization (loop over each data)
    ## calendar arbitrage
    calendar_arbi_count = 0
    calendar_loss = torch.tensor(0.)
    butterfly_loss = torch.tensor(0.)
    for elem in X_train_scaled:
        y= model.forward(elem)
        dydx = grad(y, elem, create_graph = True)[0]
        dydT = dydx[T_loc]
        if dydT < 0.0:
            calendar_arbi_count += 1
            calendar_loss += torch.exp(-dydT) * 1e-1
    loss += calendar_loss
    ## butterfly arbitrage
    butterfly_count = 0
    dydk = dydx[k_loc] # dCdk w.r.t. log-strike 
    d2ydk2 = grad(dydx[k_loc],elem, create_graph = True)[0][k_loc] # d2Cdk2 w.r.t. log-strike
    # butterfly arbitrage: d2CdK2 = 1/K^2(d2Cdk2 - dCdk) > 0, d2Cdk2 > dCdk
    # dCdK = 1/K*dcdk
    # d2CdK2 = 1/e^2k * (d2Cdk2 - dCdk)
    butter_ineq = (d2ydk2 - dydk)/torch.exp(2.0*elem[k_loc])
    if butter_ineq < 0.0: # violation of butterfly arbitrage
        butterfly_count += 1
        butterfly_loss += torch.exp(-butter_ineq) * 1e-1    
    loss += butterfly_loss

    # regularizations
    for param in model.parameters():
        l2_reg += torch.norm(param)
        loss += l2_lambda * l2_reg
    loss_arr.append(loss)
    if calendar_arbi_count > 0: print('calendar ',i, ' ',calendar_arbi_count,' ',calendar_loss)
    if butterfly_count > 0: print('butterfly ',i, ' ',butterfly_count,' ',butterfly_loss)
 
    if i % 500 == 0:
        print(f'Epoch: {i} Loss: {loss}')
 
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
if save_net: torch.save(model.state_dict(), "heston_NN") # save the model
"""